# Lighting Wildfires

#### Dependencies

In [1]:
from config import gkey
import gmaps
import numpy as np
import pandas as pd
import requests
from datetime import datetime as dt
from datetime import timedelta as td
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

## Create DataFrame from csv 
Selecting perferred column to speed up compute time

In [2]:
# Raw Data
raw_data = pd.read_csv("./Wildfire Data/FW_Veg_Rem_Combined.csv")
print(raw_data.shape)

# Only selecting the columns that i want
df1 = raw_data[['fire_size', 'fire_size_class', 'stat_cause_descr', 'latitude', 'longitude', 'state', 'disc_date_final', 'cont_date_final' ]]
print(df1.shape)

# Filtering for 'lighting'
df2 = df1.loc[df1['stat_cause_descr'] == 'Lightning',:].copy()
print(df2.shape)

(55367, 43)
(55367, 8)
(8218, 8)


## Converting date columns to datetime
Creating putout_days columns by taking disc_date_final from cont_date_final

In [3]:
df2['disc_date_final'] = pd.to_datetime(df2['disc_date_final']).dt.date.copy()
df2['cont_date_final'] = pd.to_datetime(df2['cont_date_final']).dt.date.copy()
df2['putout_days'] = df2['cont_date_final'] - df2['disc_date_final']
df2.count()

fire_size           8218
fire_size_class     8218
stat_cause_descr    8218
latitude            8218
longitude           8218
state               8218
disc_date_final     6497
cont_date_final     6181
putout_days         6165
dtype: int64

## Dropping all row with NULL/NAT value in the datetime columns

In [4]:
df3 = df2
df3.dropna(inplace=True)
df3.count()

fire_size           6165
fire_size_class     6165
stat_cause_descr    6165
latitude            6165
longitude           6165
state               6165
disc_date_final     6165
cont_date_final     6165
putout_days         6165
dtype: int64

## Saving to CSV

In [8]:
df3.to_csv("./resources - kdmcc/assets/dataKd/lightingFireRaw.csv", index=False)

In [11]:
# Renaming columns
df4 = df3.rename(columns={
    "fire_size": "Fire Size" ,
    "fire_size_class": "Fire Size Class",
    "stat_cause_descr": "Cause Stat",
    "latitude": "Lat",
    "longitude": "Lng",
    "state": "State",
    "disc_date_final": "Reported Date",
    "cont_date_final": "Resolved Date",
    "putout_days": "Days to Resolve" 
})
df4

,Fire Size,Fire Size Class,Cause Stat,Lat,Lng,State,Reported Date,Resolved Date,Days to Resolve
36,1420.0,F,Lightning,33.241800,-104.912200,NM,1994-07-02,1994-07-03,1 days
40,5.0,B,Lightning,40.834700,-112.806400,UT,2006-08-14,2006-08-15,1 days
64,1.5,B,Lightning,39.205000,-105.188333,CO,1996-07-31,1996-08-01,1 days
79,2.5,B,Lightning,42.364444,-112.232222,ID,2006-07-25,2006-07-26,1 days
84,108.0,D,Lightning,63.851700,-150.354300,AK,1997-07-01,1997-07-27,26 days
...,...,...,...,...,...,...,...,...,...
55327,4472.0,F,Lightning,47.334000,-108.364200,MT,2015-08-15,2015-08-22,7 days
55328,20621.0,G,Lightning,44.319170,-118.102500,OR,2015-08-14,2015-09-28,45 days
55329,49100.0,G,Lightning,44.505000,-117.643060,OR,2015-08-11,2015-10-21,71 days
55330,9211.0,G,Lightning,44.237000,-118.992700,OR,2015-08-12,2015-11-05,85 days


# Heatmap of Fire Size

In [12]:
# Configure gmaps with API key
gmaps.configure(api_key=gkey)

- Convert Lat and Lng into a varable named locations cast datatype as float
- Convert Fire Size into a varable named fire_size cast datatype as float

In [16]:
# Store 'Lat' and 'Lng' into  locations 
locations = df4[["Lat", "Lng"]].astype(float)

# Convert Fire Size to float and store
fire_size = df4["Fire Size"].astype(float)